In [20]:
#!pip install boto3
import boto3

In [21]:
s3 = boto3.resource('s3',
                   aws_access_key_id = 'enter-your-access-key',
                   aws_secret_access_key = 'enter-your-secret-access-key'
)

In [22]:
try:
    s3.create_bucket(Bucket = 'amichaelbucket', 
                     CreateBucketConfiguration = {
                         'LocationConstraint': 'us-west-2'}
    )
    print('bucket creation was successful')
except:
    print('maybe the bucket name or region is not good')

maybe the bucket name or region is not good


In [23]:
bucket = s3.Bucket("amichaelbucket")

In [24]:
bucket.Acl().put(ACL = 'public-read')

{'ResponseMetadata': {'RequestId': '6WDSSQB39C7EPHTJ',
  'HostId': '0lRRuO7XATAdT4yrNQlcbI2Nix7B3PrPc7DdYSJio+S2ZDBbRxB/A+6WfNEwLwne6FyVB8nveo8=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '0lRRuO7XATAdT4yrNQlcbI2Nix7B3PrPc7DdYSJio+S2ZDBbRxB/A+6WfNEwLwne6FyVB8nveo8=',
   'x-amz-request-id': '6WDSSQB39C7EPHTJ',
   'date': 'Mon, 15 Mar 2021 03:39:13 GMT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [25]:
body = open('test.txt', 'rb')

In [26]:
o = s3.Object('amichaelbucket', 'exp1test').put(Body = body)

In [27]:
s3.Object('amichaelbucket', 'exp1test').Acl().put(ACL = 'public-read')

{'ResponseMetadata': {'RequestId': 'KR7T9GE0H0TM6DCG',
  'HostId': 'pOkCS65lPKsX/a1fOxBMDEiBzzJ91hqYzTVDY1LH8vER5jkdUbfZYiMhOTNaRM2wHhUmXGggxCM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'pOkCS65lPKsX/a1fOxBMDEiBzzJ91hqYzTVDY1LH8vER5jkdUbfZYiMhOTNaRM2wHhUmXGggxCM=',
   'x-amz-request-id': 'KR7T9GE0H0TM6DCG',
   'date': 'Mon, 15 Mar 2021 03:39:14 GMT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [28]:
dyndb = boto3.resource('dynamodb',
                      region_name = 'us-west-2',
                      aws_access_key_id = 'enter-your-access-key',
                      aws_secret_access_key = 'enter-your-secret-access-key'
)

In [29]:
try:
    table = dyndb.create_table(
            TableName = 'ExperimentDataTable',
            KeySchema = [
                {
                    'AttributeName': 'PartitionKey',
                    'KeyType': 'HASH'
                },
                {
                    'AttributeName': 'RowKey',
                    'KeyType': 'RANGE'
                }
            ],
            AttributeDefinitions = [
                {
                    'AttributeName': 'PartitionKey',
                    'AttributeType': 'S'
                },
                {
                    'AttributeName': 'RowKey',
                    'AttributeType': 'S'
                }
            ],
            ProvisionedThroughput = {
                'ReadCapacityUnits': 5,
                'WriteCapacityUnits': 5
            }
    )
except:
    table = dyndb.Table("ExperimentDataTable")

In [30]:
table.meta.client.get_waiter('table_exists').wait(TableName = 'ExperimentDataTable')

In [31]:
print(table.item_count)

0


In [32]:
import csv

In [33]:
with open(r'C:\Users\andre\Desktop\Spring2021\CloudComp\cs1660-cloudstorageHW\experiments.csv', 'r', encoding="utf-8") as csvfile:
    csvf = csv.reader(csvfile, delimiter = ',', quotechar = '|')
    for item in csvf:
        print(item)
        if '.csv' in str(item[4]):
            body = open('C:\\Users\\andre\\Desktop\\Spring2021\\CloudComp\\cs1660-cloudstorageHW\\datafiles\\'+item[4],'rb')
            s3.Object('amichaelbucket', item[4]).put(Body = body)
            md = s3.Object('amichaelbucket', item[4]).Acl().put(ACL='public-read')
            url = "https://s3-us-west-2.amazonaws.com/amichaelbucket/"+item[4]
            metadata_item = {
                'PartitionKey': item[0],
                'RowKey': item[1],
                'description': item[4],
                'date': item[2],
                'url': url
            }
        try:
            table.put_item(Item = metadata_item)
        except:
            print('item may already be here, or some other failure')

['partition', 'itemid', 'date', 'comment', 'url']
item may already be here, or some other failure
['experiment1', 'data1', '2/13/2021', 'Words..', 'exp1.csv']
['experiment2', 'data2', '2/20/2021', 'second experiment', 'exp2.csv']


In [34]:
response = table.get_item(
    Key = {
        'PartitionKey': 'experiment2',
        'RowKey': 'data2'
    }
)
item = response['Item']
print(item)

{'PartitionKey': 'experiment2', 'RowKey': 'data2', 'date': '2/20/2021', 'description': 'exp2.csv', 'url': 'https://s3-us-west-2.amazonaws.com/amichaelbucket/exp2.csv'}


In [35]:
response

{'Item': {'PartitionKey': 'experiment2',
  'RowKey': 'data2',
  'date': '2/20/2021',
  'description': 'exp2.csv',
  'url': 'https://s3-us-west-2.amazonaws.com/amichaelbucket/exp2.csv'},
 'ResponseMetadata': {'RequestId': 'QI3BB828D8H8NCOOOGFADLIRFJVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 15 Mar 2021 03:39:15 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '197',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'QI3BB828D8H8NCOOOGFADLIRFJVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2455662085'},
  'RetryAttempts': 0}}